In [117]:
import geopandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import laspy
from shapely.geometry import box, Polygon, MultiPolygon, GeometryCollection
import uuid
import scipy.stats
import seaborn as sns
import math

In [118]:
inventory: pd.DataFrame = pd.read_csv(
    "C:/Users/joaov/Documents/UFMG/TCC/Dataset/DUC_A01_2016_inventory.csv",
    encoding="ISO-8859-1",
)

In [119]:
inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868
2,DUC_A01,2,002a,mapatirana,Pourouma minor,Urticaceae,31.5,O,False,NaN,20160824,840126.4377,9674180.705
3,DUC_A01,2,002b,abiurana,Pouteria gongrijpii,Sapotaceae,39.2,O,False,NaN,20160824,840109.4829,9674171.803
4,DUC_A01,2,003a,roxinho,Peltogyne lecointei,Fabaceae,10.0,O,False,NaN,20160824,840125.6847,9674181.313


In [120]:
inventory["scientific.name"].unique(), len(inventory["scientific.name"].unique())

(array(['Licania guianensis', 'Sacoglottis guianensis', 'Pourouma minor',
        'Pouteria gongrijpii', 'Peltogyne lecointei', nan,
        'Protium paniculatum', 'Qualea paraensis', 'Mouriri collocarpa',
        'Eschweilera ovata', 'Pouteria sp.', 'Ormosia grossa',
        'Ocotea cernua', 'Eperua bijuga', 'Brosimum rubescens',
        'Oenocarpus bataua', 'Hevea brasiliensis', 'Inga thibaudiana',
        'Eriotheca globosa', 'Iryanthera paraensis', 'Dipteryx odorata',
        'Sterculia pruriens', 'Miconia poeppigii', 'Sloanea guianensis',
        'Poeppigia procera', 'Xylopia sp.', 'Neea oppositifolia',
        'Geissospermum sericeum', 'Goupia glabra', 'Eschweilera micrantha',
        'Inga alba', 'Diospyros artanthifolia', 'Eschweilera coriacea',
        'Micrandra minor', 'Trichilia micrantha',
        'Tetragastris panamensis', 'Ormosia coccinea', 'Guarea guidonia',
        'Couepia robusta', 'Pouteria guianensis', 'Ocotea canaliculata',
        'Licania octandra', 'Guatteria 

In [121]:
inventory["family.name"].unique(), len(inventory["family.name"].unique())

(array(['Chrysobalanaceae', 'Humiriaceae', 'Urticaceae', 'Sapotaceae',
        'Fabaceae', nan, 'Burseraceae', 'Vochysiaceae', 'Melastomataceae',
        'Lecythidaceae', 'Lauraceae', 'Moraceae', 'Arecaceae',
        'Euphorbiaceae', 'Malvaceae', 'Myristicaceae', 'Elaeocarpaceae',
        'Annonaceae', 'Nyctaginaceae', 'Apocynaceae', 'Goupiaceae',
        'Ebenaceae', 'Meliaceae', 'Violaceae', 'Olacaceae',
        'Caryocaraceae', 'Combretaceae', 'Loganiaceae', 'Myrtaceae',
        'Simaroubaceae', 'Rubiaceae', 'Bignoniaceae', 'Malpighiaceae',
        'Salicaceae', 'Boraginaceae', 'Anacardiaceae', 'Lamiaceae',
        'Ochnaceae', 'Clusiaceae', 'Calophyllaceae'], dtype=object),
 40)

In [122]:
wood_density_dataframe = pd.read_excel("C:/Users/joaov/Documents/UFMG/TCC/Dataset/GlobalWoodDensityDatabase.xls", sheet_name="Data")

wood_density_dataframe.head()

,Number,Family,Binomial,"Wood density (g/cm^3), oven dry mass/fresh volume",Region,Reference Number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


In [123]:
wood_density_dataframe = wood_density_dataframe.rename(
    {
        "Binomial": "scientific.name",
        "Wood density (g/cm^3), oven dry mass/fresh volume": "wood.density",
        "Reference Number": "reference.number",
    },
    axis=1,
)

wood_density_dataframe.head()

,Number,Family,scientific.name,wood.density,Region,reference.number
0,1,Fabaceae,Abarema jupunba,0.78000,South America (tropical),114
1,2,Fabaceae,Abarema jupunba,0.66000,South America (tropical),198
2,3,Fabaceae,Abarema jupunba,0.55104,South America (tropical),52
3,4,Fabaceae,Abarema jupunba,0.53382,South America (tropical),65
4,5,Fabaceae,Abarema jupunba,0.55104,South America (tropical),189


In [124]:
inventory = inventory.merge(wood_density_dataframe.drop(["Region", "Number", "Family"], axis=1), on="scientific.name", how="left")

# wood_density_dataframe["reference.number"].apply(str)

inventory.head()

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,wood.density,reference.number
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN
1,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.90750,185.0
2,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.73000,64.0
3,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.97000,15.0
4,DUC_A01,2,001b,uxirana,Sacoglottis guianensis,Humiriaceae,43.6,O,False,NaN,20160824,840098.9678,9674177.868,0.82656,30.0


In [125]:
len(inventory["family.name"].unique())

40

In [128]:
inventory[inventory["reference.number"].isna()]

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,wood.density,reference.number
0,DUC_A01,2,001a,macucu,Licania guianensis,Chrysobalanaceae,49.6,O,False,NaN,20160824,840127.0705,9674180.998,NaN,NaN
20,DUC_A01,2,003b,morta,NaN,NaN,47.5,O,True,3.0,20160824,840108.5953,9674164.922,NaN,NaN
38,DUC_A01,2,006b,tamaguare,NaN,NaN,49.4,O,False,NaN,20160824,840098.7990,9674162.781,NaN,NaN
39,DUC_A01,2,007a,guajara branco,Pouteria sp.,Sapotaceae,43.8,O,False,NaN,20160824,840129.2892,9674167.459,NaN,NaN
40,DUC_A01,2,007b,tento,Ormosia grossa,Fabaceae,58.0,O,False,NaN,20160824,840115.3264,9674157.288,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4758,DUC_A01,20,044a,morta,NaN,NaN,17.0,O,True,2.0,20160830,840069.0174,9674396.727,NaN,NaN
4759,DUC_A01,20,045a,faveira rosa,NaN,NaN,23.0,O,False,NaN,20160830,840075.7151,9674388.818,NaN,NaN
4760,DUC_A01,20,046a,liana,NaN,NaN,15.7,L,False,NaN,20160830,840082.2913,9674392.692,NaN,NaN
4761,DUC_A01,20,047a,taguari,NaN,NaN,12.3,O,False,NaN,20160830,840072.2268,9674389.845,NaN,NaN


In [133]:
null_inventory_dataframe = inventory[inventory["reference.number"].isna()]

null_inventory_dataframe["common.name"].unique()

array(['macucu', 'morta', 'tamaguare', 'guajara branco', 'tento',
       'ucuuba da varzea', 'pataua', 'carana', 'fava rosa', 'inga branco',
       'pincel de macaco', 'envira', 'breu brrote', 'louro cheiroso',
       'abacabeira', 'jarana vermelha', 'taguari', 'tanenbuco amarelo',
       'acaizeiro', 'liana', 'ucuuba vermelha', 'catauari',
       'faveira amargosa', 'embauba branca', 'zaguari', 'faveira',
       'inga amarelo', 'acai', 'virola/ucuuba preta',
       'muiracatiara rajada', 'abiu rosadinha', 'mututi', 'burra leteira',
       'cucutitiriba', 'abirana', 'murta', 'gaivota', 'ata preta',
       'abirana branca', 'cacau do mato', 'carape', 'abiu preto',
       'mirindiba', 'taquari', 'breu vermelho', 'pedra ume',
       'fava folha fina', 'brteu branco', 'timbora', 'mamominho',
       'envira cunauaru', 'ucuuba folha grande', 'tanenbuco',
       'sucupira amarela', 'taruma', 'ucuuba branca', 'faveira rosa',
       'pente de macaco', 'nao identificada', 'guajara vermelho',
   

In [136]:
null_inventory_dataframe[null_inventory_dataframe["common.name"] == "faveira"] # Parkia platycephala

,area,plot,tree,common.name,scientific.name,family.name,DBH,type,Dead,D.class,date,UTM.Easting,UTM.Northing,wood.density,reference.number
567,DUC_A01,5,020a,faveira,Parkia sp.,Fabaceae,16.5,O,False,NaN,20160830,841259.5417,9674444.634,NaN,NaN
